In [12]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

location = "./clothing_data_processed.csv"
clothing_data = pd.read_csv(location)
clothing_data

,fit,user_id,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,bust_size,bust_type
0,fit,420272,2260466,137.000000,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,M,28.0,34.0,d
1,fit,273551,153475,132.000000,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,M,36.0,34.0,b
2,fit,360448,1063761,137.391709,10.0,party,This hugged in all the right places! It was a ...,petite,It was a great time to celebrate the (almost) ...,shift,64.0,XS,32.0,34.0,c
3,fit,909926,126335,135.000000,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,S,34.0,34.0,c
4,fit,151944,616682,145.000000,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,M,27.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,2252812,140.000000,10.0,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,69.0,S,42.0,34.0,dd
192540,fit,118398,682043,100.000000,10.0,work,The pattern contrast on this dress is really s...,petite,LOVE it!,dress,61.0,XS,29.0,32.0,c
192541,fit,47002,683251,135.000000,6.0,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",dress,68.0,S,31.0,36.0,a
192542,fit,961120,126335,165.000000,10.0,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,dress,66.0,L,31.0,36.0,c


In [13]:
clothing_data = clothing_data.drop(['user_id', 'review_text', 'review_summary'], axis=1)
clothing_data = clothing_data.astype({'bust_size': 'str'})
clothing_data

,fit,item_id,weight,rating,rented for,body type,category,height,size,age,bust_size,bust_type
0,fit,2260466,137.000000,10.0,vacation,hourglass,romper,68.0,M,28.0,34.0,d
1,fit,153475,132.000000,10.0,other,straight & narrow,gown,66.0,M,36.0,34.0,b
2,fit,1063761,137.391709,10.0,party,petite,shift,64.0,XS,32.0,34.0,c
3,fit,126335,135.000000,8.0,formal affair,pear,dress,65.0,S,34.0,34.0,c
4,fit,616682,145.000000,10.0,wedding,athletic,gown,69.0,M,27.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,2252812,140.000000,10.0,work,hourglass,jumpsuit,69.0,S,42.0,34.0,dd
192540,fit,682043,100.000000,10.0,work,petite,dress,61.0,XS,29.0,32.0,c
192541,fit,683251,135.000000,6.0,everyday,straight & narrow,dress,68.0,S,31.0,36.0,a
192542,fit,126335,165.000000,10.0,wedding,pear,dress,66.0,L,31.0,36.0,c


In [14]:
import collections

def get_profile(group_by_column, clothing_data_df):
    grouped_df = clothing_data_df.groupby([group_by_column])

    values = collections.defaultdict(list) 
    values_from_get_dummies = collections.defaultdict(list)
    for key, item in grouped_df:
        # print (key)
        # print(grouped_df.get_group(key), "\n\n")
        item_data = grouped_df.get_group(key)
        # print (item_data.columns)
        # print (item_data['weight'].mean())
        # break
        values[group_by_column].append(key)
        values['weight'].append(item_data['weight'].mean()) 
        values['rating'].append(item_data['rating'].mean()) 
        values['height'].append(item_data['height'].mean()) 
        values['age'].append(item_data['age'].mean()) 
        total_length = len(item_data) 
        for col in item_data.columns:
            if col in values:
                continue 
            total = sum(item_data[col])
            if total > 1:
                values_from_get_dummies[col].append(1)
            else:
                values_from_get_dummies[col].append(0)
      # break

    profile_data = pd.DataFrame()
    profile_data[group_by_column] = values[group_by_column]
    profile_data['weight'] = values['weight']
    profile_data['rating'] = values['rating']
    profile_data['height'] = values['height']
    profile_data['age'] = values['age']
    for col in values_from_get_dummies:
        profile_data[col] = values_from_get_dummies[col]

    return profile_data

In [25]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
X = clothing_data.copy()
X_object = X.select_dtypes('object')
ohe.fit(X_object)
codes = ohe.transform(X_object).toarray()
codes

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [26]:
feature_names = ohe.get_feature_names(['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type'])

clothing_data_for_item_profile = pd.concat([X.select_dtypes(exclude='object'), 
                                            pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)
clothing_data_for_item_profile
# item_profile = get_profile('item_id', clothing_data_for_item_profile)
# item_profile

,item_id,weight,rating,height,age,fit_fit,fit_large,fit_small,rented for_date,rented for_everyday,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,2260466,137.000000,10.0,68.0,28.0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,153475,132.000000,10.0,66.0,36.0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1063761,137.391709,10.0,64.0,32.0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,126335,135.000000,8.0,65.0,34.0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,616682,145.000000,10.0,69.0,27.0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,2252812,140.000000,10.0,69.0,42.0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
192540,682043,100.000000,10.0,61.0,29.0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
192541,683251,135.000000,6.0,68.0,31.0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
192542,126335,165.000000,10.0,66.0,31.0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [19]:
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity

def find_jaccard_similarity(item_profile, user_profile):
    item_id = []
    score = []
    for index, row in item_profile.iterrows():
        item_id.append(row['item_id'])
        del row['item_id']
        item_values = list(row.values)
        score.append(jaccard_score(item_values, user_profile, average='micro'))
        # break

    jaccard_similarity_scores = pd.DataFrame()
    jaccard_similarity_scores['item_id'] = item_id
    jaccard_similarity_scores['jaccard_score'] = score
    return jaccard_similarity_scores



def find_cosine_similarity(item_profile, user_profile):
    item_id = []
    score = []
    for index, row in item_profile.iterrows():
        item_id.append(row['item_id'])
        del row['item_id']
        score.append(cosine_similarity([row], [user_profile])[0][0])

    cosine_similarity_scores = pd.DataFrame()
    cosine_similarity_scores['item_id'] = item_id
    cosine_similarity_scores['cosine_similarity'] = score
    return cosine_similarity_scores


In [20]:
# Cosine and jaccard 

continuous_values = item_profile[['item_id', 'weight', 'height', 'age']].copy()
discrete_values = item_profile.drop(['weight', 'rating', 'height', 'age'], axis = 1)

# ['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type']
text_data = list(ohe.transform([['large', 'work', 'hourglass', 'blouse', 'L', '40.0', 'd']]).toarray()[0])
jaccard_similarity_scores = find_jaccard_similarity(discrete_values, text_data)
jaccard_similarity_scores


# ['weight', 'height', 'age']
numeric_data = [150, 69, 24]
cosine_similarity_scores = find_cosine_similarity(continuous_values, numeric_data)
cosine_similarity_scores
all_scores = jaccard_similarity_scores.join(cosine_similarity_scores.set_index('item_id'), on='item_id')
all_scores['average'] = all_scores.apply(lambda x : (x['jaccard_score'] + x['cosine_similarity']) / 2, axis = 1)
item_ratings = item_profile[['item_id', 'rating']].copy()
all_scores = all_scores.join(item_ratings.set_index('item_id'), on='item_id')
all_scores = all_scores.sort_values(by=['cosine_similarity', 'rating'], ascending = False)

all_scores

,item_id,jaccard_score,cosine_similarity,average,rating
5706,2895264,0.831325,0.999998,0.915662,10.000000
1228,711336,0.831325,0.999969,0.915647,10.000000
1570,882528,0.831325,0.999969,0.915647,10.000000
4080,2099557,0.831325,0.999969,0.915647,10.000000
271,248794,0.707865,0.999956,0.853911,9.333333
...,...,...,...,...,...
166,204824,0.831325,0.968249,0.899787,10.000000
4181,2147735,0.831325,0.968249,0.899787,8.000000
5751,2918861,0.831325,0.964345,0.897835,10.000000
4242,2180111,0.831325,0.963014,0.897170,4.000000


In [21]:
# ['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type']
text_data = list(ohe.transform([['large', 'work', 'hourglass', 'blouse', 'L', '40.0', 'd']]).toarray()[0])
# ['weight', 'height', 'age']
numeric_data = [150, 69, 24]
user_data = numeric_data + text_data

item_data = item_profile.drop(['rating'], axis = 1)

cosine_similarity_scores = find_cosine_similarity(item_data, user_data)
cosine_similarity_scores
item_ratings = item_profile[['item_id', 'rating']].copy()
all_scores = cosine_similarity_scores.join(item_ratings.set_index('item_id'), on='item_id')
all_scores = all_scores.sort_values(by=['cosine_similarity', 'rating'], ascending = False)

all_scores

,item_id,cosine_similarity,rating
5706,2895264.0,0.999872,10.0
1228,711336.0,0.999844,10.0
1570,882528.0,0.999844,10.0
4080,2099557.0,0.999844,10.0
5366,2731234.0,0.999815,10.0
...,...,...,...
166,204824.0,0.968127,10.0
4181,2147735.0,0.968127,8.0
5751,2918861.0,0.964224,10.0
4242,2180111.0,0.962893,4.0


In [30]:
clothing_data_new = clothing_data.copy()
clothing_data_new['weight'] = clothing_data_new['weight'] // 10
clothing_data_new['age'] = clothing_data_new['age'] // 10 
clothing_data_new = clothing_data_new.astype({'weight': 'str', 'age': 'str', 'height': 'str'})
clothing_data_new

,fit,item_id,weight,rating,rented for,body type,category,height,size,age,bust_size,bust_type
0,fit,2260466,13.0,10.0,vacation,hourglass,romper,68.0,M,2.0,34.0,d
1,fit,153475,13.0,10.0,other,straight & narrow,gown,66.0,M,3.0,34.0,b
2,fit,1063761,13.0,10.0,party,petite,shift,64.0,XS,3.0,34.0,c
3,fit,126335,13.0,8.0,formal affair,pear,dress,65.0,S,3.0,34.0,c
4,fit,616682,14.0,10.0,wedding,athletic,gown,69.0,M,2.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,2252812,14.0,10.0,work,hourglass,jumpsuit,69.0,S,4.0,34.0,dd
192540,fit,682043,10.0,10.0,work,petite,dress,61.0,XS,2.0,32.0,c
192541,fit,683251,13.0,6.0,everyday,straight & narrow,dress,68.0,S,3.0,36.0,a
192542,fit,126335,16.0,10.0,wedding,pear,dress,66.0,L,3.0,36.0,c


In [35]:
ohe_new = OneHotEncoder(handle_unknown='ignore')
X_new = clothing_data_new.copy()
X_object_new = X_new.select_dtypes('object')
ohe.fit(X_object_new)
codes_new = ohe.transform(X_object_new).toarray()
feature_names_new = ohe.get_feature_names(['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type'])
items_new = pd.concat([X_new.select_dtypes(exclude='object'), 
                        pd.DataFrame(codes_new, columns = feature_names_new).astype(int)], axis=1)

items_new

,item_id,rating,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,2260466,10.0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,153475,10.0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,1063761,10.0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,126335,8.0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,616682,10.0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,2252812,10.0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
192540,682043,10.0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
192541,683251,6.0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
192542,126335,10.0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [42]:
def get_item_profile(df):
    values_from_get_dummies = collections.defaultdict(list)
    item_id = []
    grouped_items = df.groupby(['item_id'])
    for key, item in grouped_items:

        item_data = grouped_items.get_group(key)
        item_id.append(key)
        for col in item_data.columns:
            if col == 'item_id' or col == 'rating':
                continue
            total = sum(item_data[col])
            if total > 1:
                values_from_get_dummies[col].append(1)
            else:
                values_from_get_dummies[col].append(0)


    profile_data = pd.DataFrame()
    profile_data['item_id'] = item_id
    for col in values_from_get_dummies:
        profile_data[col] = values_from_get_dummies[col]
    return profile_data

In [43]:
items_profile_new = get_item_profile(items_new)
items_profile_new

,item_id,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,weight_15.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,123373,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,0
1,123793,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
2,124204,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,0
3,124553,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
4,125424,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,2963850,1,0,0,0,0,0,1,0,0,...,1,1,0,1,0,0,0,0,0,0
5846,2964470,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5847,2965009,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5848,2965924,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [47]:
# ['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type']
user_input = ['large', '15', 'work', 'hourglass', 'blouse', '69', 'L', '2', '40.0', 'd']
user_input = list(ohe.transform([user_input]).toarray()[0])
test_score = find_jaccard_similarity(items_profile_new, user_input)
# test_score
item_ratings = item_profile[['item_id', 'rating']].copy()
test_score = test_score.join(item_ratings.set_index('item_id'), on='item_id')
test_score = test_score.sort_values(by=['jaccard_score', 'rating'], ascending = False)
test_score

,item_id,jaccard_score,rating
5078,2588581,0.927007,8.000000
5236,2665954,0.913043,10.000000
2494,1324909,0.913043,8.000000
2025,1100910,0.913043,7.333333
163,203040,0.899281,10.000000
...,...,...,...
1,123793,0.320000,9.537923
14,130259,0.320000,8.769248
5,125465,0.313433,9.370328
20,132738,0.287805,9.232277
